# Network Expansion and Topology Modification

Demonstrates how to programmatically modify system topology by tapping existing lines and splitting buses. This is essential for planning studies where new substations or interconnections are evaluated.

Import the case and instantiate the `GridWorkBench`.

```python
from esapp import GridWorkBench
from esapp.components import *
from esapp.saw._helpers import create_object_string

wb = GridWorkBench(case_path)
```

In [1]:
# This cell is hidden in the documentation.
from esapp import GridWorkBench
from esapp.components import *
from esapp.saw._helpers import create_object_string
import numpy as np
import matplotlib.pyplot as plt
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 3.9239 sec


In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_spy_matrices

## Y-Bus Before Modification

Capture the Y-Bus sparsity pattern before making network changes:

In [2]:
Y_before = wb.ybus()
print(f"Y-Bus before: {Y_before.shape}, nnz={Y_before.nnz}")

Y-Bus before: (37, 37), nnz=153


## Tap Existing Transmission Lines

Select a branch to tap and insert a new bus at the midpoint:

In [3]:
branches = wb[Branch]
b = branches.iloc[10]
tobus = b['BusNum']
frombus = b['BusNum:1']
circuit = b['LineCircuit']
branch_str = create_object_string("Branch", tobus, frombus, circuit)

Tap the transmission line at 50% of its length and create a new bus at the tap point:

In [4]:
new_bus_num = int(wb[Bus, "BusNum"]["BusNum"].max()) + 100

wb.esa.TapTransmissionLine(
    branch_str,
    50.0,
    new_bus_num,
    'CAPACITANCE',
    False, False,
    'Tapped_Substation'
)

## Split a Bus

Split an existing bus into two buses connected by a tie-line:

In [5]:
target_bus = 1
split_bus_num = int(wb[Bus, 'BusNum']['BusNum'].max()) + 1

wb.esa.SplitBus(
    create_object_string("Bus", target_bus),
    split_bus_num,
    insert_tie=True,
    line_open=False,
    branch_device_type="Breaker"
)

## Validate Network Changes

Compare the Y-Bus before and after the topology modifications:

In [ ]:
V = wb.pflow()
Y_after = wb.ybus()

print(f'Y-Bus before: {Y_before.shape}, nnz={Y_before.nnz}')
print(f'Y-Bus after:  {Y_after.shape}, nnz={Y_after.nnz}')
print(f'New buses added: {Y_after.shape[0] - Y_before.shape[0]}')

plot_spy_matrices([Y_before, Y_after],
                  [f'Y-Bus Before\n{Y_before.shape}', f'Y-Bus After\n{Y_after.shape}'],
                  colors=['steelblue', 'tomato'])